In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from mpmath_integration import quad_phi

In [3]:
instance = 'instances/instances_beta_eq_zero_algo.csv'
# instance = 'instances/instances_beta_eq_zero_asymp_xmu.csv'

In [4]:
df = pd.read_csv(instance)

In [5]:
df

,x,alpha,mu,delta,mpmath,scipy,gnp
0,0.118216,27.116783,-2.158274,5.764235,9.999994e-01,9.9999940889103400E-01,9.999994e-01
1,4.504637,48.394205,-4.901985,43.929992,1.000000e+00,NAN,1.000000e+00
2,-3.558404,35.239417,-3.896193,14.967645,6.979935e-01,6.9799347240368237E-01,6.979935e-01
3,4.486494,27.261715,-3.687490,37.296710,1.000000e+00,NAN,1.000000e+00
4,-1.881685,23.184287,-0.677947,46.868379,1.985413e-01,NAN,1.985413e-01
...,...,...,...,...,...,...,...
995,0.654838,18.843563,-3.865584,25.968663,9.999383e-01,9.9993827933742463E-01,9.999383e-01
996,4.374292,14.346248,4.788995,14.979224,3.421822e-01,3.4218224649992784E-01,3.421822e-01
997,-4.812338,29.124041,0.421437,23.416039,3.229725e-09,3.2298373920161157E-09,3.229725e-09
998,3.656635,40.191090,-3.878165,22.050325,1.000000e+00,NAN,1.000000e+00


In [6]:
df.loc[df['scipy'] == 'NAN', 'scipy'] = np.NaN
df['scipy'] = df['scipy'].astype(float)

df.loc[df['gnp'] == 'NAN', 'gnp'] = np.NaN
df['gnp'] = df['gnp'].astype(float)

In [7]:
df['relerr_gnp'] = np.absolute(df['gnp'] / df['mpmath'] - 1)
df['abs_gnp'] = np.absolute(df['gnp'] - df['mpmath'])

In [8]:
df['relerr_scipy'] = np.absolute(df['scipy'] / df['mpmath'] - 1)
df['abs_scipy'] = np.absolute(df['scipy'] - df['mpmath'])

In [9]:
df['da'] = df['delta'] * df['alpha']

In [10]:
df[['x', 'alpha', 'mu', 'delta', 'da']].describe()

,x,alpha,mu,delta,da
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.028046,24.988738,-0.184133,25.244619,629.850878
std,2.899992,14.212944,2.935010,14.576032,556.461781
min,-4.979432,0.005802,-4.992318,0.019839,0.072838
25%,-2.404761,12.245459,-2.796726,13.053924,165.740970
50%,-0.062113,25.312001,-0.240109,25.633686,477.202961
75%,2.628567,36.824296,2.387672,37.761555,940.623336
max,4.991993,49.989557,4.963131,49.975786,2429.390376


In [11]:
mask_inf = df['relerr_gnp'] == np.inf
mask_nan = (df['relerr_gnp'].isna() & (df['mpmath'] != df['gnp']) & (df['mpmath'] != 0))

mask_err = (df['relerr_gnp'] > 5e-13) | mask_nan

In [12]:
mask_inf.sum(), mask_nan.sum()

(0, 0)

In [13]:
np.count_nonzero(~mask_err)

979

In [14]:
df[mask_inf]

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da


In [15]:
df[mask_nan].describe()

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
mask_err.sum(), (df['relerr_gnp'] > 1e-11).sum()

(21, 5)

In [17]:
df[mask_err].sort_values('relerr_gnp', ascending=False).head(10)

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da
126,-3.902653,44.211944,4.424909,2.630028,2.784465e-119,4.191759e-120,4.191734e-120,8.494600e-01,2.365291e-119,8.494591e-01,2.365289e-119,116.278665
284,-4.679187,8.105449,4.426663,47.481199,8.882387e-05,8.882387e-05,8.882387e-05,7.345657e-11,6.524702e-15,1.221245e-14,1.084202e-18,384.856410
900,-4.890191,8.106770,3.087431,42.076074,2.414964e-04,2.414964e-04,2.414964e-04,5.980316e-11,1.444225e-14,3.020362e-12,7.293970e-16,341.101065
226,-2.236870,28.294808,-1.737694,0.387034,6.643482e-05,6.643482e-05,6.643482e-05,4.823764e-11,3.204658e-15,3.108624e-15,2.032879e-19,10.951055
376,-3.774743,10.234563,1.813893,41.467037,2.780292e-03,2.780292e-03,2.780292e-03,1.110489e-11,3.087461e-14,1.798561e-14,4.987330e-17,424.396998
528,-0.863043,8.307507,4.802118,47.055974,8.695583e-03,8.695583e-03,8.695583e-03,2.812861e-12,2.445960e-14,2.220446e-15,1.908196e-17,390.917839
428,0.333262,26.689436,1.403701,5.113588,7.364576e-03,7.364576e-03,7.364576e-03,2.392309e-12,1.761872e-14,9.769963e-15,7.199102e-17,136.478776
201,-1.122309,8.965739,4.787624,46.422097,4.738531e-03,4.738531e-03,4.738531e-03,2.227996e-12,1.055753e-14,2.242651e-14,1.066855e-16,416.208425
259,-0.853498,5.360744,4.889100,36.142301,1.358843e-02,1.358843e-02,1.358843e-02,1.233680e-12,1.676263e-14,1.065814e-14,1.457168e-16,193.749622
925,-3.698604,32.160454,-2.859044,3.198198,4.030350e-03,4.030350e-03,4.030350e-03,1.110889e-12,4.477321e-15,1.820766e-14,7.372575e-17,102.855501


In [18]:
df[~mask_err].sort_values('relerr_gnp', ascending=False).head()

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da
107,-3.037031,41.299345,-0.401018,22.003783,0.000155,NaN,0.000155,4.907186e-13,7.619231e-17,NaN,NaN,908.741830
260,-0.367600,23.493288,2.029883,10.157619,0.000143,0.000143,0.000143,3.712586e-13,5.323433e-17,4.123368e-13,5.911612e-17,238.635869
918,2.864853,43.486137,1.535583,25.495426,0.958721,NaN,0.958721,3.324008e-13,3.186340e-13,NaN,NaN,1108.697579
314,-1.007090,48.058820,-2.268491,24.298380,0.961965,NaN,0.961965,3.286260e-13,3.160805e-13,NaN,NaN,1167.751471
708,-1.662676,36.835187,-0.483371,6.022829,0.001821,0.001821,0.001821,3.268497e-13,5.952270e-16,6.994405e-15,1.279359e-17,221.852040


In [19]:
df[~mask_err].describe()

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da
count,979.000000,979.000000,979.000000,979.000000,9.790000e+02,6.240000e+02,9.790000e+02,9.790000e+02,9.790000e+02,6.240000e+02,6.240000e+02,979.000000
mean,0.077905,25.125793,-0.238400,25.140504,5.265632e-01,5.180563e-01,5.265632e-01,1.690351e-14,3.858170e-15,3.307628e-03,3.205128e-03,634.187940
std,2.892679,14.183404,2.924451,14.510410,4.340391e-01,4.316984e-01,4.340391e-01,4.770632e-14,1.762236e-14,5.656882e-02,5.656840e-02,558.010372
min,-4.979432,0.005802,-4.992318,0.019839,7.993726e-89,7.993726e-89,7.993726e-89,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.072838
25%,-2.367384,12.666383,-2.840251,13.052507,1.062619e-02,8.512719e-03,1.062619e-02,4.440892e-16,7.453890e-20,1.554312e-15,5.697482e-17,163.988361
50%,0.003564,25.477654,-0.317505,25.477747,6.037489e-01,5.628716e-01,6.037489e-01,1.998401e-15,3.330669e-16,6.272760e-15,6.661338e-16,489.342139
75%,2.687271,36.925378,2.348648,37.443067,9.990252e-01,9.990377e-01,9.990252e-01,8.937295e-15,1.554312e-15,2.098322e-14,3.219647e-15,948.045322
max,4.991993,49.989557,4.963131,49.975786,1.000000e+00,1.000000e+00,1.000000e+00,4.907186e-13,3.186340e-13,1.000000e+00,1.000000e+00,2429.390376


In [20]:
df[mask_err].describe()

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da
count,21.000000,21.000000,21.000000,21.000000,2.100000e+01,1.800000e+01,2.100000e+01,2.100000e+01,2.100000e+01,1.800000e+01,1.800000e+01,21.000000
mean,-2.296334,18.599400,2.345714,30.098383,2.273798e-02,4.841146e-03,2.273798e-02,4.045047e-02,1.970923e-14,4.719217e-02,1.224403e-16,427.661190
std,2.259957,14.468655,2.281729,17.063181,5.756796e-02,5.891144e-03,5.756796e-02,1.853674e-01,3.675449e-14,2.002194e-01,1.717898e-16,443.725244
min,-4.890191,5.155281,-2.859044,0.387034,2.784465e-119,4.191759e-120,4.191734e-120,5.297984e-13,2.365291e-119,2.220446e-15,2.365289e-119,10.951055
25%,-3.918405,8.105449,0.802670,15.514064,1.867355e-04,1.559886e-04,1.867355e-04,6.539214e-13,7.623025e-16,1.171285e-14,4.767779e-17,183.710878
50%,-3.350061,10.234563,2.867143,36.142301,2.780292e-03,2.409777e-03,2.780292e-03,1.021627e-12,6.524702e-15,1.920686e-14,7.285839e-17,341.101065
75%,-0.863043,28.294808,4.424909,42.225241,8.848200e-03,8.362831e-03,8.848200e-03,2.812861e-12,1.676263e-14,2.400857e-13,1.056013e-16,416.208425
max,3.312748,48.526817,4.889100,49.327802,1.973718e-01,1.922320e-02,1.973718e-01,8.494600e-01,1.360856e-13,8.494591e-01,7.293970e-16,1642.339210


In [21]:
idx = 2
df[mask_err].iloc[idx].mpmath, df[mask_err].iloc[idx].gnp

(2.784464701295552e-119, 4.1917341879812364e-120)

In [ ]:
df[mask_err]

In [ ]:
x, alpha, mu, delta = df[mask_err].iloc[idx][['x', 'alpha', 'mu', 'delta']]
x, alpha, mu, delta

In [ ]:
mp_result = quad_phi(x, alpha, 0, mu, delta, digits=300)
mp_result

#### 1. Decision tree (test for more complex cases)

In [21]:
from sklearn import tree

In [22]:
df['target'] = (~mask_err).astype(int)

In [23]:
df.head()

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da,target
0,0.118216,27.116783,-2.158274,5.764235,0.999999,0.999999,0.999999,0.000000e+00,0.000000e+00,6.883383e-15,6.772360e-15,156.307509,1
1,4.504637,48.394205,-4.901985,43.929992,1.000000,NaN,1.000000,0.000000e+00,0.000000e+00,NaN,NaN,2125.957034,1
2,-3.558404,35.239417,-3.896193,14.967645,0.697993,0.697993,0.697993,2.708944e-14,1.887379e-14,1.465494e-14,1.021405e-14,527.451083,1
3,4.486494,27.261715,-3.687490,37.296710,1.000000,NaN,1.000000,8.881784e-16,7.771561e-16,NaN,NaN,1016.772283,1
4,-1.881685,23.184287,-0.677947,46.868379,0.198541,NaN,0.198541,2.353673e-14,4.662937e-15,NaN,NaN,1086.609971,1


In [24]:
df['omega'] = np.sqrt(df['delta'] ** 2 + (df['x'] - df['mu']) ** 2)
df['aw'] = df['alpha'] * df['omega']
df['xmu'] = df['x'] - df['mu']
df['xmu2'] = df['xmu'] ** 2
df['aow'] = df['alpha'] / df['omega']
df['woa'] = df['omega'] / df['alpha']
df['da'] = df['delta'] * df['alpha']

In [25]:
df['method'] = 'integration'

criteria_asymp_xmu = (df['xmu2'] >= 70) & (df['aow'] >= 1.0)
df.loc[criteria_asymp_xmu, 'method'] = 'asymptotic_xmu'

criteria_asymp_a = (df['xmu2'] <= 2.5) & (df['da'] >= 150) & (df['alpha'] >= 5)
df.loc[criteria_asymp_a, 'method'] = 'asymptotic_alpha'

criteria1 = (df['xmu'].abs() <= 20) & (df['aow'] <= 0.25)
criteria2 = (df['xmu2'] <= 1.25) & (df['aw'] <= 750)

df.loc[criteria1 | criteria2, 'method'] = 'series'

In [26]:
df[df['target'] == 1].head(10)

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da,target,omega,aw,xmu,xmu2,aow,woa,method
0,0.118216,27.116783,-2.158274,5.764235,9.999994e-01,9.999994e-01,9.999994e-01,0.000000e+00,0.000000e+00,6.883383e-15,6.772360e-15,156.307509,1,6.197484,168.055841,2.276490,5.182409,4.375450,0.228548,integration
1,4.504637,48.394205,-4.901985,43.929992,1.000000e+00,NaN,1.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,2125.957034,1,44.925814,2174.149035,9.406622,88.484543,1.077203,0.928330,asymptotic_xmu
2,-3.558404,35.239417,-3.896193,14.967645,6.979935e-01,6.979935e-01,6.979935e-01,2.708944e-14,1.887379e-14,1.465494e-14,1.021405e-14,527.451083,1,14.971456,527.585385,0.337789,0.114102,2.353774,0.424850,series
3,4.486494,27.261715,-3.687490,37.296710,1.000000e+00,NaN,1.000000e+00,8.881784e-16,7.771561e-16,NaN,NaN,1016.772283,1,38.181914,1040.904476,8.173984,66.814017,0.713996,1.400569,integration
4,-1.881685,23.184287,-0.677947,46.868379,1.985413e-01,NaN,1.985413e-01,2.353673e-14,4.662937e-15,NaN,NaN,1086.609971,1,46.883835,1086.968295,-1.203738,1.448985,0.494505,2.022225,asymptotic_alpha
5,-0.766736,23.320484,2.758345,17.745632,2.848206e-05,2.848206e-05,2.848206e-05,1.110223e-15,3.388132e-20,9.436896e-15,2.676624e-19,413.836732,1,18.092365,421.922698,-3.525081,12.426193,1.288968,0.775814,integration
6,3.277026,34.100154,-0.956142,46.567891,9.998524e-01,NaN,9.998524e-01,0.000000e+00,0.000000e+00,NaN,NaN,1587.972261,1,46.759899,1594.519776,4.233168,17.919715,0.729261,1.371252,integration
7,-0.908009,11.726034,-3.539573,44.111371,9.126371e-01,9.126371e-01,9.126371e-01,3.330669e-16,3.330669e-16,9.992007e-15,9.214851e-15,517.251432,1,44.189797,518.171061,2.631564,6.925129,0.265356,3.768520,integration
8,0.495937,35.770341,-3.320786,6.081767,1.000000e+00,1.000000e+00,1.000000e+00,2.220446e-16,2.220446e-16,2.886580e-15,2.886580e-15,217.546895,1,7.180200,256.838199,3.816723,14.567376,4.981803,0.200731,integration
9,-4.724409,24.549996,4.528849,6.392469,2.901707e-54,2.901811e-54,2.901707e-54,1.776357e-15,5.216071e-69,3.605106e-05,1.046096e-58,156.935077,1,11.246619,276.104448,-9.253258,85.622783,2.182878,0.458111,asymptotic_xmu


In [27]:
x, alpha, mu, delta = df[df['target'] == 1].head(1)[['x', 'alpha', 'mu', 'delta']].values[0]

In [28]:
x, alpha, mu, delta

(0.1182162470025671,
 27.116782747705887,
 -2.1582741889630075,
 5.764235016147439)

In [29]:
df['method'].value_counts()

method
integration         609
series              239
asymptotic_alpha    139
asymptotic_xmu       13
Name: count, dtype: int64

In [30]:
df.groupby('method')[['target', 'relerr_gnp']].describe()

target                                               \
                  count      mean       std  min  25%  50%  75%  max   
method                                                                 
asymptotic_alpha  139.0  0.978417  0.145842  0.0  1.0  1.0  1.0  1.0   
asymptotic_xmu     13.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0   
integration       609.0  0.993432  0.080844  0.0  1.0  1.0  1.0  1.0   
series            239.0  0.941423  0.235325  0.0  1.0  1.0  1.0  1.0   

                 relerr_gnp                                                 \
                      count          mean           std  min           25%   
method                                                                       
asymptotic_alpha      139.0  5.163975e-14  1.060019e-13  0.0  4.829470e-15   
asymptotic_xmu         13.0  5.790240e-15  1.451931e-14  0.0  0.000000e+00   
integration           609.0  1.394844e-03  3.442185e-02  0.0  3.330669e-16   
series                239.0  8.820991e-13  6.878793e-12  0.0  8.881784e-16   

                                                            
                           50%           75%           max  
method                                                      
asymptotic_alpha  1.487699e-14  5.040413e-14  7.058798e-13  
asymptotic_xmu    0.000000e+00  0.000000e+00  4.818368e-14  
integration       1.110223e-15  3.774758e-15  8.494600e-01  
series            4.884981e-15  1.959544e-14  7.345657e-11

In [31]:
df_integration = df[mask_err & (df['method'] == 'integration')]
df_integration.describe()

,x,alpha,mu,delta,mpmath,scipy,gnp,relerr_gnp,abs_gnp,relerr_scipy,abs_scipy,da,target,omega,aw,xmu,xmu2,aow,woa
count,4.000000,4.000000,4.000000,4.000000,4.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,4.000000,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,-3.453646,28.394480,1.583180,19.703249,1.098452e-04,8.421514e-05,1.098452e-04,2.123650e-01,6.826747e-17,2.831530e-01,5.892639e-17,570.899785,0.0,21.508326,643.276825,-5.036827,29.042712,1.902716,0.948408
std,0.959465,16.495463,2.212487,15.601544,7.895031e-05,7.354129e-05,7.895031e-05,4.247300e-01,4.641905e-17,4.904354e-01,5.157325e-17,720.487732,0.0,13.525717,675.597740,2.213019,26.963432,2.109890,0.524028
min,-4.372821,11.841570,-0.839616,2.630028,2.784465e-119,4.191759e-120,4.191734e-120,5.297984e-13,2.365291e-119,5.961898e-13,2.365289e-119,116.278665,0.0,8.733003,190.312947,-8.327562,12.483537,0.736801,0.197526
25%,-4.020195,15.472253,0.392098,12.293055,8.765943e-05,5.843962e-05,8.765943e-05,5.646039e-13,5.866889e-17,7.081002e-13,4.046785e-17,166.852825,0.0,14.236948,308.596806,-5.229216,15.667609,0.783953,0.790660
50%,-3.648209,28.762203,1.373714,17.985438,1.263227e-04,1.168792e-04,1.263227e-04,6.984413e-13,8.706821e-17,8.200107e-13,8.093569e-17,262.490631,0.0,18.466644,367.063900,-4.143270,17.169510,0.905718,1.119443
75%,-3.081660,41.684431,2.564796,25.395632,1.485085e-04,1.263227e-04,1.485085e-04,2.123650e-01,9.666679e-17,4.247295e-01,8.838958e-17,666.537591,0.0,25.738022,701.743918,-3.950881,30.544614,2.024481,1.277191
max,-2.145348,44.211944,4.424909,40.212090,1.867355e-04,1.357662e-04,1.867355e-04,8.494600e-01,9.893345e-17,8.494591e-01,9.584347e-17,1642.339210,0.0,40.367012,1648.666554,-3.533205,69.348292,5.062628,1.357218


In [ ]:
df_integration.sort_values('relerr_gnp', ascending=True).head(50)

In [ ]:
df_integration.head(10)

In [ ]:
idx = 0

In [ ]:
df_integration[df_integration.index == 820]

In [ ]:
df_integration.iloc[idx][['mpmath', 'scipy', 'gnp']].values

In [ ]:
x, alpha, mu, delta = df_integration.iloc[idx][['x', 'alpha', 'mu', 'delta']].values

In [ ]:
x, alpha, mu, delta = df_integration[df_integration.index == 820][['x', 'alpha', 'mu', 'delta']].values[0]

In [ ]:
list(df_integration[df_integration.index == 820][['mpmath', 'scipy', 'gnp']].values[0])

In [ ]:
mp_result = quad_phi(x, alpha, 0, mu, delta, digits=250)
mp_result

In [ ]:
for k, v in zip(['x', 'alpha', 'mu', 'delta'], (x, alpha, mu, delta)):
    print(f'double {k} = {v};')

In [ ]:
df[(df['xmu'].abs() > 4) & (df['aw'] > 100)]

In [ ]:
X = df.copy()
y = df['target']

X = X[['xmu2', 'woa', 'aw', 'da', 'alpha']]

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=2)
clf = clf.fit(X, y)

In [ ]:
clf.score(X, y)

In [ ]:
tree.plot_tree(clf, feature_names=X.columns, filled=True)

In [ ]:
mask_series = (df['alpha'] <= 2.425) & (df['mu'] <= 4.406)
# mask_series = ((X['xmu2'] <= 20) & (X['aow'] <= 0.25)) | ((X['xmu2'] <= 1.25) & (X['aw'] <= 750))
mask_series = (X['xmu2'] >= 70) & (X['woa'] <= 1)

In [ ]:
X.describe()

In [ ]:
df[mask_series].describe()

In [ ]:
df[mask_series][df.loc[mask_series, 'relerr_gnp'].isna()]

In [ ]:
df[X['aw'] > 750].describe()